In [3]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf
import happybase
import warnings
import sklearn.model_selection
import sklearn.preprocessing
import csv
import json
import pickle

In [4]:
data_dir =  r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"

In [5]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

['/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000000_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000002_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000001_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000003_0']


In [6]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list
old_df = copy.deepcopy(df)

b'Skipping line 461777: expected 32 fields, saw 33\n'
<string>:2: DtypeWarning: Columns (1,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1min, sys: 3.21 s, total: 1min 3s
Wall time: 1min 3s


In [7]:
df = copy.deepcopy(old_df)

In [8]:
# 获得指定列
use_column_list = ["用户id","用户出生时间","关注的作者","关注的马甲"]
y_col = "用户出生时间"
df = df[use_column_list]

In [9]:
df.fillna("",inplace=True)
df

,用户id,用户出生时间,关注的作者,关注的马甲
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1
...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1


In [10]:
def deal_with_birth_date(birth_date):
    if isinstance(birth_date,int) or (not birth_date.split("-")[0]) or (int(birth_date.split("-")[0]) < 1940):
        return False
    return True

predict_df = df[-df["用户出生时间"].apply(deal_with_birth_date)]
train_df = df[df["用户出生时间"].apply(deal_with_birth_date)]

In [11]:
predict_df

,用户id,用户出生时间,关注的作者,关注的马甲
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1
...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1


In [12]:
train_df

,用户id,用户出生时间,关注的作者,关注的马甲
10739,10025,1988-10-28,-1,-1
11009,100346,1989-0-0,-1,-1
11857,10065,1985-1-1,-1,-1
12227,100782,1988-10-0,-1,-1
12469,10087,1971-2-0,-1,-1
...,...,...,...,...
14637137,98608,2013-1-1,-1,-1
14642274,99083,1991-5-0,-1,-1
14643212,99274,1987-2-0,-1,-1
14643469,99300,1989-5-0,-1,-1


## 数据分析（可以略过）

In [ ]:
temp_list = []
for i in user_df['用户出生时间']:
    if (not i.split("-")[0]) or (int(i.split("-")[0]) < 1940):
        temp_list.append(i)
len(temp_list)

In [ ]:
#查看信息
user_df.info()

In [ ]:
user_df['用户姓名'].unique()

In [ ]:
(user_df['用户姓名']=="-1").sum()

In [ ]:
#男，女，默认，缺失 -1
user_df['用户出生时间'].unique()

In [ ]:
user_df['微信网名'].unique()

In [ ]:
user_df['关注的作者'].unique()

In [ ]:
(user_df['关注的作者']=="-1").sum()

In [ ]:
temp_list = []
for _str in user_df['关注的作者'].fillna(""):
    for _id in _str.split(","):
        temp_list.append(_id)

In [ ]:
collections.Counter(temp_list)

In [ ]:
#查看行业信息
user_df['关注的马甲'].unique()

In [ ]:
(user_df['关注的马甲']=="-1").sum()

In [ ]:
user_df["文章频道"].unique()

In [ ]:
temp_list = []
for _str in user_df["文章频道"].fillna(""):
    for v in _str.split(","):
        temp_list.append(v)
collections.Counter(temp_list)

In [ ]:
temp_list = []
for _str in user_df["文章类型"].fillna(""):
    for v in _str.split(","):
        temp_list.append(v)
collections.Counter(temp_list)

In [ ]:
user_df.fillna("",inplace=True)

In [ ]:
user_df["关注的作者"]
user_df["关注的马甲"]
user_df["文章频道"]
user_df["浏览文章时间"]
user_df["文章类型"]
user_df["文章图片数量"]
user_df["文章字数"]

In [ ]:
user_df

In [ ]:
user_df["文章字数"][0]

In [145]:
def temp_function(time_str):
    if not time_str:
        return ""
    time_str_split = time_str.split(",")
    hour_list = []
    for _time in time_str_split:
        hour_list.append(datetime.datetime.strptime(_time, '%Y-%m-%d %H:%M:%S').hour)
    return hour_list
user_df["浏览文章时间"] = user_df["浏览文章时间"].apply(lambda x : temp_function(x))

0                   [8, 11, 16, 9, 19, 8, 18, 17, 14, 21]
1               [8, 11, 8, 13, 8, 23, 11, 20, 18, 15, 12]
2                                [18, 18, 18, 18, 18, 18]
3       [23, 22, 23, 12, 19, 6, 19, 18, 8, 6, 7, 13, 1...
4                               [18, 21, 8, 9, 21, 21, 8]
                              ...                        
2211                                                 [10]
2212                     [14, 14, 14, 16, 19, 14, 14, 14]
2213                                      [9, 9, 9, 9, 9]
2214                                         [13, 13, 13]
2215                                         [12, 10, 10]
Name: 浏览文章时间, Length: 2216, dtype: object

In [ ]:
user_df.info()

# 模型训练

In [11]:
#训练数据
multi_hot_column_odict = collections.OrderedDict()
multi_hot_column_set = set([v for v in use_column_list if v != y_col and v!="用户id"])
for i,column in enumerate(multi_hot_column_set):
    multi_hot_column_odict[column] = sklearn.preprocessing.MultiLabelBinarizer()
    split_column_list = [item.split(",") if isinstance(item,str) else str(item) for item in train_df[column]]
    train_df[column] = multi_hot_column_odict[column].fit_transform(split_column_list).tolist()

/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
train_df[y_col].unique()

array(['1988-10-28', '1989-0-0', '1985-1-1', ..., '1976-10-29',
       '1979-3-0', '1978-8-23'], dtype=object)

In [13]:
train_df

,用户id,用户出生时间,关注的作者,关注的马甲
10739,10025,1988-10-28,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11009,100346,1989-0-0,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11857,10065,1985-1-1,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12227,100782,1988-10-0,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12469,10087,1971-2-0,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
14637137,98608,2013-1-1,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14642274,99083,1991-5-0,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14643212,99274,1987-2-0,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14643469,99300,1989-5-0,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
def temp_function(x):
    x = datetime.datetime.now().year - int(x.split("-")[0])
    y = 0
    if x < 20:
        y = "~20"
    elif x<25:
        y = "21~25"
    elif x<29:
        y = "26~29"
    elif x<35:
        y = "30~35"
    elif x<45:
        y = "36~45"
    elif x<50:
        y = "46~50"
    else:
        y = "50~"
    return y    

train_df[y_col] = train_df[y_col].apply(lambda x : temp_function(x))

/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
train_df[y_col].unique()

array(['30~35', '36~45', '46~50', '~20', '26~29', '21~25', '50~'],
      dtype=object)

In [16]:
%%time
y_onehot = sklearn.preprocessing.LabelBinarizer()
y_onehot.fit(train_df[y_col])
train_df[y_col] = y_onehot.transform(train_df[y_col]).tolist()

CPU times: user 309 ms, sys: 3.06 ms, total: 312 ms
Wall time: 459 ms


/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
train_df[y_col]

10739       [0, 0, 1, 0, 0, 0, 0]
11009       [0, 0, 1, 0, 0, 0, 0]
11857       [0, 0, 0, 1, 0, 0, 0]
12227       [0, 0, 1, 0, 0, 0, 0]
12469       [0, 0, 0, 0, 1, 0, 0]
                    ...          
14637137    [0, 0, 0, 0, 0, 0, 1]
14642274    [0, 0, 1, 0, 0, 0, 0]
14643212    [0, 0, 1, 0, 0, 0, 0]
14643469    [0, 0, 1, 0, 0, 0, 0]
14643950    [0, 0, 0, 1, 0, 0, 0]
Name: 用户出生时间, Length: 60312, dtype: object

In [22]:
for k,v in multi_hot_column_odict.items():
    print(k,len(v.classes_))

关注的马甲 1676
关注的作者 11


In [24]:
train, test = sklearn.model_selection.train_test_split(train_df, test_size=0.2)
train, val = sklearn.model_selection.train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

38599 train examples
9650 validation examples
12063 test examples


In [25]:
def generate_xy(data_df, y_col, multi_hot_column_odict, shuffle=True):
    """"""
    x_df = data_df.copy()
    x_list = []
    y_list = []
    for index in x_df.index:
        temp_x_list = np.concatenate(x_df.loc[index,[column for column in multi_hot_column_odict]]).tolist()
        temp_y_list = x_df.loc[index,y_col]
        x_list.append(temp_x_list)
        y_list.append(temp_y_list)
    return np.array(x_list,dtype=np.float32), np.array(y_list,dtype=np.int32)

In [26]:
%%time
x_train,y_train = generate_xy(train,y_col,multi_hot_column_odict)
x_valid,y_valid = generate_xy(val,y_col,multi_hot_column_odict)
x_test,y_test = generate_xy(test,y_col,multi_hot_column_odict)

CPU times: user 57.6 s, sys: 322 ms, total: 58 s
Wall time: 56.9 s


In [28]:
input_dim = 0
for k,v in  multi_hot_column_odict.items():
    input_dim += len(v.classes_)
input_dim

1687

In [29]:
y_dim = len(y_onehot.classes_)
y_dim

7

In [43]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(input_dim,)))
model.add(tf.keras.layers.Dense(y_dim, activation='softmax',use_bias=True,
                                #kernel_regularizer=tf.keras.regularizers.l1(0.01)
                               ))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [44]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 7)                 11816     
Total params: 11,816
Trainable params: 11,816
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.fit(x_train, 
          y_train, 
          batch_size=64,
          epochs=15,
         validation_data = (x_valid,y_valid))

Epoch 1/15
604/604 [==============================] - 1s 1ms/step - loss: 1.1670 - categorical_accuracy: 0.5262 - val_loss: 1.2555 - val_categorical_accuracy: 0.5115
Epoch 2/15
604/604 [==============================] - 1s 1ms/step - loss: 1.1665 - categorical_accuracy: 0.5267 - val_loss: 1.2570 - val_categorical_accuracy: 0.5107
Epoch 3/15
604/604 [==============================] - 1s 1ms/step - loss: 1.1661 - categorical_accuracy: 0.5265 - val_loss: 1.2578 - val_categorical_accuracy: 0.5113
Epoch 4/15
604/604 [==============================] - 1s 1ms/step - loss: 1.1657 - categorical_accuracy: 0.5267 - val_loss: 1.2591 - val_categorical_accuracy: 0.5108
Epoch 5/15
604/604 [==============================] - 1s 1ms/step - loss: 1.1654 - categorical_accuracy: 0.5264 - val_loss: 1.2603 - val_categorical_accuracy: 0.5106
Epoch 6/15
604/604 [==============================] - 1s 1ms/step - loss: 1.1650 - categorical_accuracy: 0.5269 - val_loss: 1.2614 - val_categorical_accuracy: 0.5105
Epoc

In [51]:
model.evaluate(x_test,y_test)

377/377 [==============================] - 0s 775us/step - loss: 1.3063 - categorical_accuracy: 0.5080


[1.306262493133545, 0.5079996585845947]

In [54]:
#保存模型
with open("multi_hot_column_odict.pkl","wb") as f:
    pickle.dump(multi_hot_column_odict,f)
with open("y_onehot.pkl","wb") as f:
    pickle.dump(y_onehot,f)
model.save("model_output")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_output/assets


In [55]:
multi_hot_column_odict

OrderedDict([('关注的马甲', MultiLabelBinarizer()),
             ('关注的作者', MultiLabelBinarizer())])

In [56]:
train_df

,用户id,用户出生时间,关注的作者,关注的马甲
10739,10025,"[0, 0, 1, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11009,100346,"[0, 0, 1, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11857,10065,"[0, 0, 0, 1, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12227,100782,"[0, 0, 1, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12469,10087,"[0, 0, 0, 0, 1, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
14637137,98608,"[0, 0, 0, 0, 0, 0, 1]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14642274,99083,"[0, 0, 1, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14643212,99274,"[0, 0, 1, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14643469,99300,"[0, 0, 1, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [23]:
import happybase
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(train_df.index):
            batch.put(str(train_df.loc[index,"用户id"]).encode(), {"attribute:input_age".encode(): y_onehot.inverse_transform(np.array([train_df.loc[index,y_col]]))[0].encode()})
            #print(str(train_df.loc[index,"用户id"]).encode(),{"attribute:input_age".encode(): y_onehot.inverse_transform(np.array([train_df.loc[index,"用户出生时间"]]))[0].encode()})
            #break

,用户id,用户出生时间,关注的作者,关注的马甲
10739,10025,"[0, 0, 1, 0, 0, 0, 0]",-1,-1
11009,100346,"[0, 0, 1, 0, 0, 0, 0]",-1,-1
11857,10065,"[0, 0, 0, 1, 0, 0, 0]",-1,-1
12227,100782,"[0, 0, 1, 0, 0, 0, 0]",-1,-1
12469,10087,"[0, 0, 0, 0, 1, 0, 0]",-1,-1
...,...,...,...,...
14637137,98608,"[0, 0, 0, 0, 0, 0, 1]",-1,-1
14642274,99083,"[0, 0, 1, 0, 0, 0, 0]",-1,-1
14643212,99274,"[0, 0, 1, 0, 0, 0, 0]",-1,-1
14643469,99300,"[0, 0, 1, 0, 0, 0, 0]",-1,-1
